# Project Reacher 

In this short Notebook, I will show how my implementation of Deep Deterministic Policy Gradient (DDPG) manages to solve Unity's Reacher environment. This project was carried out as part as Udacity's DRL nanodegree and some of the code are inspired from examples of the course.  


In [ ]:
%matplotlib inline

# Import
from unityagents import UnityEnvironment
import numpy as np

We will start by launching the environment. 

1) Download the environment 
Linux: [click here](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P2/Reacher/Reacher_Linux.zip)
Mac OSX: [click here](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P2/Reacher/Reacher.app.zip)
Windows (32-bit): [click here](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P2/Reacher/Reacher_Windows_x86.zip)
Windows (64-bit): [click here](https://s3-us-west-1.amazonaws.com/udacity-drlnd/P2/Reacher/Reacher_Windows_x86_64.zip)

2) Indicate the location of the following file : 
Mac: "path/to/Reacher.app"

Windows (x86): "path/to/Reacher_Windows_x86/Reacher.exe"

Windows (x86_64): "path/to/Reacher_Windows_x86_64/Reacher.exe"

Linux (x86): "path/to/Reacher_Linux/Reacher.x86"

Linux (x86_64): "path/to/Reacher_Linux/Reacher.x86_64"

Linux (x86, headless): "path/to/Reacher_Linux_NoVis/Reacher.x86"

Linux (x86_64, headless): "path/to/Reacher_Linux_NoVis/Reacher.x86_64"


env = UnityEnvironment(file_name="Reacher.app")

In [ ]:
#Indicate the environment location in the location variable
location = '../Project/Reacher_Linux/Reacher.x86_64'
#no_graphics=True disables the graphics and speeds up the training
env = UnityEnvironment(file_name=location,no_graphics=False)

# Start the Brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

Finally, let's have a look at the environment

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

## Create the agent

We now know that the state size is 33 and that the environment is waiting for a 4x1 dimensional action. We can use this information to create our agent.

In [ ]:
from ddpg_agent import Agent
from collections import deque
import torch
import matplotlib.pyplot as plt

#Create an agent
agent = Agent(state_size=33, action_size=4, random_seed=1)

## Training

* Note : skip this part if you don't want to train the agent, *
The agent and the descriptions of the neural networks are located in ddpg_agent.py and model.py respectively. Here, we just create one agent. 

We will start the training. We will output the average score over the last 5 episodes and the one over the 100 last ones. The environment is considered solved when the agent reaches an average score of 30 over the 100 last episodes. We will collect experience from 20 different agents, but only train one actor and one critic. Each agent will use the same weights for the training. 

In [ ]:

def ddpg(n_episodes=550, print_every=5):
    scores_last_episodes = deque(maxlen=print_every)
    scores_target = deque(maxlen=100)
    my_scores = []
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations                  # get the current state (for each agent)
        agent.reset()                                          #Reset the agent 
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        while True:
            actions = agent.act(states)              # select an action (for each agent)
            env_info = env.step(actions)[brain_name]           # send all actions to the environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            agent.step(states,actions,rewards,next_states,dones) #Memorize and learn
            scores += env_info.rewards                        # update the score (for each agent)
            states = next_states                               # roll over states to next time step            
            if np.any(dones):
                break 
        score=np.mean(scores)   
        scores_last_episodes.append(score)
        scores_target.append(score)
        my_scores.append(score)
        print('\rEpisode {}\tAverage Score over 5 last episodes: {:.2f}\t Average over 100 last episodes :{:.2f}'.format(i_episode, np.mean(scores_last_episodes),np.mean(scores_target)), end="")
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_last_episodes)))
        if(np.mean(scores_target)>30):
            print('Environment Solved !')
            break
        
    return my_scores

fig = plt.figure()

scores = ddpg()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()



Let's watch our agent !

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]
states = env_info.vector_observations                  # get the current state (for each agent)
        #agent.reset()
scores = np.zeros(num_agents)         # initialize the score (for each agent)
while True:
    actions = agent.act(states,True)              # select an action (for each agent)
    env_info = env.step(actions)[brain_name]           # send all actions to the environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    agent.step(states,actions,rewards,next_states,dones) #Memorize and learn
    scores += env_info.rewards                        # update the score (for each agent)
    states = next_states                               # roll over states to next time step            
    if np.any(dones):
        break 
print(np.mean(scores))   


Do not forget to save the weights !

In [ ]:
#Save the critic
torch.save(agent.critic_local.state_dict(), "critic.pth")
#Save the actor
torch.save(agent.actor_local.state_dict(),"actor.pth")


## Watching a trained agent the weights 

In [ ]:
# Load and watch
agent.critic_local.load_state_dict(torch.load("critic.pth"))
agent.actor_local.load_state_dict(torch.load("actor.pth"))
agent.critic_target.load_state_dict(torch.load("critic.pth"))
agent.actor_target.load_state_dict(torch.load("actor.pth"))

agent.epsilon=0
agent.reset()

env_info = env.reset(train_mode=False)[brain_name]
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)         # initialize the score (for each agent)
while True:
    actions = agent.act(states,False)              # select an action (for each agent)
    env_info = env.step(actions)[brain_name]           # send all actions to the environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                        # update the score (for each agent)
    states = next_states                               # roll over states to next time step            
    if np.any(dones):
        break 
print(np.mean(scores))   